In [ ]:
import pandas as pd

sales_doc = "../doccuments/sales_and_eodStocks.xlsx"
transactions_doc = "../doccuments/transactions.xlsx"

sales_df = pd.read_excel(sales_doc)
transactions_doc = pd.read_excel(transactions_doc)